# Flowers Image Classification with TensorFlow on Cloud ML Engine

This notebook demonstrates how to do image classification from scratch on a flowers dataset using Estimator/Experiment.

In [18]:
import os
PROJECT = 'cloud-training-demos' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'cloud-training-demos-ml' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
MODEL_TYPE = 'cnn'

# do not change these
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['MODEL_TYPE'] = MODEL_TYPE

In [2]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


## Input functions to read JPEG images

The key difference between this notebook and [the MNIST one](./mnist_models.ipynb) is in the input function.
In the input function here, we are doing the following:
* Reading JPEG images, rather than 2D integer arrays.
* Reading in batches of batch_size images (see the parameter to reader.read) rather than slicing our in-memory structure to be batch_size images.
* Resizing the images to the expected HEIGHT, WIDTH. Because this is a real-world dataset, the images are of different sizes. We need to preprocess the data to, at the very least, resize them to constant size.

In [15]:
%writefile /tmp/test.csv
gs://cloud-ml-data/img/flower_photos/daisy/754296579_30a9ae018c_n.jpg,daisy
gs://cloud-ml-data/img/flower_photos/dandelion/18089878729_907ed2c7cd_m.jpg,dandelion
gs://cloud-ml-data/img/flower_photos/dandelion/284497199_93a01f48f6.jpg,dandelion

Overwriting /tmp/test.csv


In [16]:
import tensorflow as tf
def read_lines(filename):
    from tensorflow.python.lib.io.file_io import FileIO
    with FileIO(filename, 'rb') as f:
      lines = f.readlines()
      return [line.rstrip() for line in lines]   

with tf.Session() as sess:
  #csvfile = 'gs://cloud-ml-data/img/flower_photos/train_set.csv'
  csvfile = '/tmp/test.csv'
  lines = read_lines(csvfile)
  print lines[:2]
  filename, label = tf.decode_csv(lines, record_defaults=[['a'],['b']])
  print filename[1].eval()
  print label[1].eval()


['gs://cloud-ml-data/img/flower_photos/daisy/754296579_30a9ae018c_n.jpg,daisy', 'gs://cloud-ml-data/img/flower_photos/dandelion/18089878729_907ed2c7cd_m.jpg,dandelion']
gs://cloud-ml-data/img/flower_photos/dandelion/18089878729_907ed2c7cd_m.jpg
dandelion


## Run as a Python module

Let's run it as Python module.  Note the --model parameter

In [13]:
%bash
rm -rf flowersmodel.tar.gz flowers_trained
export PYTHONPATH=${PYTHONPATH}:${PWD}/flowersmodel
python -m trainer.task --help

usage: task.py [-h] [--train_batch_size TRAIN_BATCH_SIZE]
               [--learning_rate LEARNING_RATE] [--train_steps TRAIN_STEPS]
               --output_dir OUTPUT_DIR --model MODEL [--job-dir JOB_DIR]
               [--ksize1 KSIZE1] [--ksize2 KSIZE2] [--nfil1 NFIL1]
               [--nfil2 NFIL2] [--dprob DPROB] [--batch_norm]

optional arguments:
  -h, --help            show this help message and exit
  --train_batch_size TRAIN_BATCH_SIZE
                        Batch size for training steps
  --learning_rate LEARNING_RATE
                        Initial learning rate for training
  --train_steps TRAIN_STEPS
                        Steps to run the training job for. A step is one
                        batch-size,
  --output_dir OUTPUT_DIR
                        GCS location to write checkpoints and export models
  --model MODEL         Type of model. Supported types are ['cnn',
                        'dnn_dropout', 'dnn', 'linear']
  --job-dir JOB_DIR     this model ignores 

Now, let's do it on ML Engine. Note the --model parameter

In [ ]:
%bash
OUTDIR=gs://${BUCKET}/flowers/trained_${MODEL_TYPE}
JOBNAME=flowers_${MODEL_TYPE}_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/flowersmodel/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC_GPU \
   --runtime-version=1.2 \
   -- \
   --output_dir=$OUTDIR \
   --train_steps=5000 --learning_rate=0.01 --train_batch_size=40 \
   --model=$MODEL_TYPE --batch_norm --augment

Here are my results:

Model | Accuracy | Time taken | Run time parameters
--- | :---: | ---
cnn with batch-norm | 0.582 | 47 min | 1000 steps, LR=0.01, Batch=40
as above, plus augment | 0.615 | 3 hr | 5000 steps, LR=0.01, Batch=40

## Deploying and predicting with model

Deploy the model:

In [ ]:
%bash
MODEL_NAME="flowers"
MODEL_VERSION=${MODEL_TYPE}
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/flowers/trained_${MODEL_TYPE}/export/Servo | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
gcloud ml-engine versions delete --quiet ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
#gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION}

To predict with the model, let's take one of the example images that is available on Google Cloud Storage <img src="http://storage.googleapis.com/cloud-ml-data/img/flower_photos/sunflowers/1022552002_2b93faf9e7_n.jpg" />

In [48]:
%writefile test.json
{"imageurl": "gs://cloud-ml-data/img/flower_photos/sunflowers/1022552002_2b93faf9e7_n.jpg"}

Overwriting test.json


Send it to the prediction service

In [58]:
%bash
gcloud ml-engine predict --model=flowers --version=${MODEL_TYPE} --json-instances=./test.json

CLASS       CLASSID  PROBABILITIES
sunflowers  3        [0.005260971374809742, 0.0005250766407698393, 0.008500268682837486, 0.9802882671356201, 0.0054253824055194855]


<pre>
# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
</pre>